In [1]:
import time 
import requests
import re
import bs4
from bs4 import BeautifulSoup, Comment  
import pandas as pd
import scipy as sc
import numpy as np

In [81]:
df = pd.read_csv('./Anime_Data_Frame')

In [82]:
df.isnull().sum()

Unnamed: 0            0
Format              130
Episodes            176
EpisodeDuration    2149
Status              130
Average Score      1582
Mean Score          162
Popularity          130
Favorites          2206
Studios            4523
Producers          3761
Source             2119
Genres              725
Romaji              130
English            4770
Native              317
Synonyms           3786
Hashtag            8492
Duration           8042
dtype: int64

In [83]:
df.Format.unique()

array([nan, 'TV', 'Special', 'TV Short', 'ONA', 'OVA', 'Movie',
       'ONA\n(Chinese)', 'Music', 'TV\n(Chinese)',
       'Movie\n(South Korean)', 'Movie\n(Chinese)', 'TV\n(South Korean)',
       'TV Short\n(South Korean)', 'ONA\n(South Korean)',
       'Movie\n(Doujin)', 'TV Short\n(Chinese)', 'OVA\n(South Korean)',
       'ONA\n(Taiwanese)', 'Movie\n\n(South Korean)',
       'Special\n(Chinese)'], dtype=object)

In [84]:
drop_list = ['Unnamed: 0','Duration', 'English','EpisodeDuration','Episodes','Studios','Hashtag','Synonyms', 'Producers','Status']
df = df.drop(drop_list, axis = 1)

In [85]:
df

,Format,Average Score,Mean Score,Popularity,Favorites,Source,Genres,Romaji,Native
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TV,66%,66%,6050.0,69.0,Manga,ComedyDramaFantasyMahou ShoujoRomance,Shin Shirayuki-hime Densetsu Pretear,新白雪姫伝説プリーティア
2,TV,71%,72%,6307.0,98.0,Manga,ActionAdventureDramaSci-Fi,Uchuu Kaizoku Captain Harlock,宇宙海賊キャプテンハーロック
3,Special,NaN,59%,103.0,NaN,NaN,ComedySupernatural,Esper Mami Special: My Angel Mami-chan,エスパー魔美スペシャル マイエンジェル魔美ちゃん
4,TV Short,59%,60%,6176.0,48.0,Manga,ComedyMechaSci-FiSlice of Life,Uchuu Senkan Tiramisu,宇宙戦艦ティラミス
...,...,...,...,...,...,...,...,...,...
9995,ONA,NaN,59%,118.0,NaN,Other,NaN,Shirotan: Shirotan ga Ippai!,しろたん しろたんがいっぱい!
9996,OVA,NaN,48%,95.0,1.0,NaN,ComedyDramaRomance,Chuuhai Lemon Love 30S,酎ハイれもん：LOVE30S
9997,Movie,65%,65%,7219.0,29.0,Original,ActionAdventureComedyFantasy,ONE PIECE 3D: Mugiwara Chase,ONE PIECE 3D 麦わらチェイス
9998,TV Short,74%,75%,8532.0,115.0,Other,ComedySlice of Life,Pingu in the City,ピングー in ザ・シティ


In [86]:

df.isnull().sum()

Format            130
Average Score    1582
Mean Score        162
Popularity        130
Favorites        2206
Source           2119
Genres            725
Romaji            130
Native            317
dtype: int64

In [25]:
df.Source.unique()

array([nan, 'Manga', 'Light Novel', 'Video Game', 'Original', 'Other',
       'Novel', 'Comic', 'Web Novel', 'Visual Novel', 'Picture Book',
       'Multimedia Project', 'Doujinshi', 'Anime', 'Game', 'Live Action'],
      dtype=object)

In [87]:
row_drop_list = ['Average Score','Mean Score','Popularity','Favorites','Genres','Romaji','Native']
small_df = df.dropna(axis = 0, subset = row_drop_list)


In [88]:
small_df.isnull().sum()

Format             0
Average Score      0
Mean Score         0
Popularity         0
Favorites          0
Source           793
Genres             0
Romaji             0
Native             0
dtype: int64

In [89]:
small_df['Source'].fillna('Other', inplace = True)
small_df.isnull().sum()

C:\Users\ItanM\Anaconda3\lib\site-packages\pandas\core\generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


Format           0
Average Score    0
Mean Score       0
Popularity       0
Favorites        0
Source           0
Genres           0
Romaji           0
Native           0
dtype: int64

['Comedy',
 'Drama',
 'Fantasy',
 'Psychological',
 'Thriller',
 'Slice of ',
 'Life']

AttributeError: 'DataFrame' object has no attribute 'Slice_of_Life'

Format           1967
Average Score    1967
Mean Score       1967
Popularity       1967
Favorites        1967
Source           1967
Genres           1967
Romaji           1967
Native           1967
Action           1250
Adventure        1494
Comedy           1010
Drama            1482
Ecchi            1766
Fantasy          1506
Horror           1878
Mahou Shoujo     1966
Mecha            1753
Music            1876
Mystery          1833
Psychology       1966
Romance          1567
Sci-Fi           1501
Slice of Life    1966
Sports           1841
Supernatural     1718
Thriller         1932
Psychological    1874
dtype: int64

In [90]:
def fix_genre(df):
    pattern = '[A-Z][^A-Z]*'
    genre_list = ['Action','Adventure','Comedy','Drama','Ecchi','Fantasy', 'Horror', 'Mahou Shoujo', 'Mecha', 'Music','Mystery'
                 ,'Psychological', 'Romance','Sci-Fi', 'Slice of Life', 'Sports','Supernatural','Thriller']
    for genre in genre_list:
        df[genre] = 0
    row_index = 1
    for genre in df['Genres']:
        check_list = re.findall(pattern, genre)
        for check in check_list:
            if(check == 'Mahou'):
                check += ' Shoujo'
            if(check == 'Sci-'):
                check += 'Fi'
            if(check == 'Slice of'):
                check += ' Life'
            for col in df:
                if(check == col):
                    df.at[row_index, col] = 1
        row_index +=1
    return df

In [97]:
count = 1
for genre in small_df['Genres']:
    count += 1
print(count)

9028


In [99]:
small_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9027 entries, 1 to 7061
Data columns (total 27 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Format         7061 non-null   object 
 1   Average Score  7061 non-null   object 
 2   Mean Score     7061 non-null   object 
 3   Popularity     7061 non-null   float64
 4   Favorites      7061 non-null   float64
 5   Source         7061 non-null   object 
 6   Genres         7061 non-null   object 
 7   Romaji         7061 non-null   object 
 8   Native         7061 non-null   object 
 9   Action         7777 non-null   float64
 10  Adventure      7533 non-null   float64
 11  Comedy         8017 non-null   float64
 12  Drama          7545 non-null   float64
 13  Ecchi          7261 non-null   float64
 14  Fantasy        7521 non-null   float64
 15  Horror         7149 non-null   float64
 16  Mahou Shoujo   7061 non-null   float64
 17  Mecha          7274 non-null   float64
 18  Music   